In [10]:
import pickle
import gzip
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import sobel
from tensorflow.keras.callbacks import ModelCheckpoint

# Load the data
def load_data():
    path_to_train_data = 'flatland_train.data'
    path_to_test_data = 'flatland_test.data'

    # Load training data
    with gzip.open(path_to_train_data, 'rb') as f:
        X_train, y_train = pickle.load(f)

    # Load testing data
    with gzip.open(path_to_test_data, 'rb') as f:
        X_test, y_test = pickle.load(f)

    return X_train, y_train, X_test, y_test

X, y, X_p, y_p = load_data()

# Normalize and convert the data to tensors
X = tf.convert_to_tensor(X, dtype=tf.float32) / 255.0
X_p = tf.convert_to_tensor(X_p, dtype=tf.float32) / 255.0

# Function to apply Sobel edge detection with padding
import numpy as np

import tensorflow as tf

def apply_sobel_edges_with_padding(images, pad_width):
    # Pad images
    padded_images = tf.pad(images, [[0, 0], [pad_width, pad_width], [pad_width, pad_width]], mode='CONSTANT')

    # Define Sobel kernels
    sobel_x = tf.constant([[1, 0, -1], [2, 0, -2], [1, 0, -1]], dtype=tf.float32)
    sobel_y = tf.constant([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], dtype=tf.float32)

    # Reshape kernels to match the expected dimensions for Conv2D
    sobel_x = tf.expand_dims(tf.expand_dims(sobel_x, axis=-1), axis=-1)  # Shape: (3, 3, 1, 1)
    sobel_y = tf.expand_dims(tf.expand_dims(sobel_y, axis=-1), axis=-1)  # Shape: (3, 3, 1, 1)

    # Apply Sobel filters using tf.nn.conv2d
    gradient_x = tf.nn.conv2d(padded_images[..., tf.newaxis], sobel_x, strides=[1, 1, 1, 1], padding='VALID')
    gradient_y = tf.nn.conv2d(padded_images[..., tf.newaxis], sobel_y, strides=[1, 1, 1, 1], padding='VALID')

    # Compute the gradient magnitude
    gradient_magnitude = tf.sqrt(tf.square(gradient_x) + tf.square(gradient_y))

    return tf.squeeze(gradient_magnitude, axis=-1)

# Example usage
pad_width = 1
X_edges = apply_sobel_edges_with_padding(X, pad_width)
X_p_edges = apply_sobel_edges_with_padding(X_p, pad_width)




# Apply Sobel edges and add channel dimension
pad_width = 1
X_edges = apply_sobel_edges_with_padding(X, pad_width)
X_p_edges = apply_sobel_edges_with_padding(X_p, pad_width)
X_edges = np.expand_dims(X_edges, axis=-1)

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_datagen.fit(X_edges)

# Split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X_edges, y, test_size=0.2, random_state=42)

# Build the model
model = keras.Sequential()
model.add(layers.Input(shape=(52, 52, 1)))

# Define filter sizes and dropout rates for each layer
layer_configs = [
    {"filters": 16, "dropout": 0.2},
    {"filters": 32, "dropout": 0.3},
    {"filters": 64, "dropout": 0.4},
    {"filters": 128, "dropout": 0.5}
]

# Add convolutional layers in a loop
for config in layer_configs:
    model.add(layers.Conv2D(config["filters"], (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(config["dropout"]))

# Fully connected layers
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Custom Callback for Printing Epoch Info
class CustomEpochLogger(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        accuracy = logs.get('accuracy')
        loss = logs.get('loss')
        print(f'Epoch {epoch + 1}: accuracy: {accuracy:.4f}, loss: {loss:.4f}')

# Model checkpointing
checkpoint_callback = ModelCheckpoint(
    filepath='model_epoch_{epoch:02d}.weights.h5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True,
    verbose=0  # Suppress model saving output
)

# Train the model
train_generator = train_datagen.flow(
    x_train,
    y_train,
    batch_size=32,
    shuffle=True
)

model.fit(
    train_generator,
    epochs=55,  # Increased epochs for better training
    validation_data=(x_test, y_test),
    callbacks=[checkpoint_callback, CustomEpochLogger()],
    verbose=0  # Suppress default logging
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'\nTest accuracy: {test_accuracy:.6f}')

# Predict on X_p_edges and display predictions
X_p_edges = np.expand_dims(X_p_edges, axis=-1)
predictions = model.predict(X_p_edges)
predicted_labels = [np.argmax(pred) for pred in predictions]
formatted_predictions = ''.join([str(round(p)) for p in predicted_labels])
print("Predicted labels:", formatted_predictions)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1: accuracy: 0.2635, loss: 2.0659
Epoch 2: accuracy: 0.3250, loss: 1.5566
Epoch 3: accuracy: 0.4009, loss: 1.3880
Epoch 4: accuracy: 0.4403, loss: 1.2831
Epoch 5: accuracy: 0.4920, loss: 1.1754
Epoch 6: accuracy: 0.5169, loss: 1.0979
Epoch 7: accuracy: 0.5641, loss: 1.0227
Epoch 8: accuracy: 0.6010, loss: 0.9600
Epoch 9: accuracy: 0.6258, loss: 0.8918
Epoch 10: accuracy: 0.6593, loss: 0.8545
Epoch 11: accuracy: 0.6858, loss: 0.7891
Epoch 12: accuracy: 0.7034, loss: 0.7672
Epoch 13: accuracy: 0.7221, loss: 0.7128
Epoch 14: accuracy: 0.7465, loss: 0.6880
Epoch 15: accuracy: 0.7684, loss: 0.6426
Epoch 16: accuracy: 0.7841, loss: 0.6110
Epoch 17: accuracy: 0.7977, loss: 0.5819
Epoch 18: accuracy: 0.8062, loss: 0.5572
Epoch 19: accuracy: 0.8217, loss: 0.5229
Epoch 20: accuracy: 0.8223, loss: 0.5125
Epoch 21: accuracy: 0.8349, loss: 0.4944
Epoch 22: accuracy: 0.8388, loss: 0.4872
Epoch 23: accuracy: 0.8519, loss: 0.4628
Epoch 24: accuracy: 0.8506, loss: 0.4550
Epoch 25: accuracy: 0.860